# Machine Learning with Spark

In [ ]:
from pydataset import data
import pyspark
import pyspark.ml
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = spark.createDataFrame(data('tips'))

train, test = df.randomSplit([0.8, 0.2], seed=123)

In [ ]:
def shape(df: pyspark.sql.DataFrame):
    return df.count(), len(df.columns)

In [ ]:
shape(train)

In [ ]:
shape(test)

## Regression

We'll first demonstrate a regression problem: predicting the tip amount.

In [ ]:
train.show(5)

`pyspark.ml.feature.RFormula`

- `tip ~ total_bill`: predict tip based on total bill
- `tip ~ total_bill + size`: predict tip based on total bill and size
- `tip ~ .`: predict tip based on all the other features in the dataset

In [ ]:
# nb: spark's rformula does encoding
rf = pyspark.ml.feature.RFormula(formula="tip ~ total_bill + size").fit(train)

rf.transform(train).show(5)

`features` and `labels` columns are the shape/name required for `pyspark.ml`

In [ ]:
train_input = rf.transform(train).select('features', 'label')
train_input.show(5)

Create, fit, and use the model.

**Note**: unlike `sklearn`, each step produces a new object!

In [ ]:
lr = pyspark.ml.regression.LinearRegression()
lr

In [ ]:
# print(lr.explainParams())

In [ ]:
lr_fit = lr.fit(train_input)
lr_fit.transform(train_input).show(5)

Training results:

In [ ]:
lr_fit.summary

In [ ]:
lr_fit.summary.r2, lr_fit.summary.rootMeanSquaredError

In [ ]:
[x for x in dir(lr_fit.summary) if not x.startswith('_')]

How do we do on the test data?

In [ ]:
test_input = rf.transform(test)
lr_fit.transform(test_input).show(4)

In [ ]:
evaluator = pyspark.ml.evaluation.RegressionEvaluator()
rmse = evaluator.evaluate(lr_fit.transform(test_input))
rmse

## Classification

Preprocess the training data

In [ ]:
rf = pyspark.ml.feature.RFormula(formula='time ~ total_bill + size').fit(train)
train_input = rf.transform(train)
train_input.show(50)

Create and fit the model

In [ ]:
lr = pyspark.ml.classification.LogisticRegression()

In [ ]:
# print(lr.explainParams())

In [ ]:
lr_fit = lr.fit(train_input)

Model Evaluation

In [ ]:
[x for x in dir(lr_fit.summary) if not x.startswith('_')]

In [ ]:
# area under TPR (recall) vs FPR (FP / (FP + TN)) curve
# https://en.wikipedia.org/wiki/Receiver_operating_characteristic
lr_fit.summary.areaUnderROC

In [ ]:
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(lr_fit.transform(rf.transform(test)))
test_auc

In [ ]:
test_input = rf.transform(test)

In [ ]:
# confusion matrix for the test data
(lr_fit.transform(test_input)
 .select('time', 'total_bill', 'size', 'label', 'probability', 'prediction')
 .groupby('prediction') # predicted == rows
 .pivot('label') # actual values are columns
 .count()
 .show())

In [ ]:
# Many other preprocessing steps
# dir(pyspark.ml.feature)